# Quickstart

This section runs through the API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

## Working with data

PyTorch has two [primitives to work with
data](https://pytorch.org/docs/stable/data.html):
`torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset`
stores the samples and their corresponding labels, and `DataLoader`
wraps an iterable around the `Dataset`.


In [33]:
import gc
import numpy as np  # type: ignore
import random
import torch  # type: ignore

from IPython.display import display
from PIL import Image  # type: ignore
from torch import nn, Tensor  # type: ignore
from torch.nn import CrossEntropyLoss  # type: ignore
from torch.nn.modules.loss import _Loss  # type: ignore
from torch.optim import SGD  # type: ignore
from torch.optim.optimizer import Optimizer  # type: ignore
from torch.utils.data import DataLoader  # type: ignore
from torchvision import datasets  # type: ignore
from torchvision.datasets import FashionMNIST  # type: ignore
from torchvision.transforms import ToTensor  # type: ignore
from typing import List

PyTorch offers domain-specific libraries such as
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which
include datasets. For this tutorial, we will be using a TorchVision
dataset.

The `torchvision.datasets` module contains `Dataset` objects for many
real-world vision data like **CIFAR**, **COCO** ([full list
here](https://pytorch.org/vision/stable/datasets.html)). In this
tutorial, we use the FashionMNIST dataset. Every TorchVision `Dataset`
includes two arguments: `transform` and `target_transform` to modify the
samples and labels respectively.


In [4]:
train_data: FashionMNIST = FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data: FashionMNIST = FashionMNIST(
    root="data", train=True, download=False, transform=ToTensor()
)

In [5]:
def display_img_with_idx(idx: int) -> None:
    img: Tensor = train_data.__getitem__(idx)[0]

    if img.is_cuda:
        img = img.cpu()

    img_arr: np.ndarray = img.numpy().transpose(1, 2, 0)
    if img_arr.max() <= 1:  # normalize
        img_arr = (img_arr * 255).astype(np.uint8)
    if img_arr.shape[2] == 1:  # grayscale
        img_arr = img_arr.squeeze(axis=2)

    img_pil: Image = Image.fromarray(img_arr)
    display(img_pil)


for _ in range(5):
    display_img_with_idx(_)

We pass the `Dataset` as an argument to `DataLoader`. This wraps an
iterable over our dataset, and supports **automatic batching**, **sampling**,
**shuffling** and **multiprocess data loading**. Here we define a batch size of
64, i.e. each element in the dataloader iterable will return a batch of
64 features and labels.


In [6]:
batch_size: int = 64

train_dataloader: DataLoader = DataLoader(train_data, batch_size=batch_size)
test_dataloader: DataLoader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


Read more about [loading data in PyTorch](data_tutorial.html).


---


# Creating Models

To define a neural network in PyTorch, we create a class that inherits
from
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and
specify how data will pass through the network in the `forward`
function. To accelerate operations in the neural network, we move it to
the GPU or MPS if available.


In [7]:
device: str = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten: nn.Flatten = nn.Flatten(start_dim=1, end_dim=-1)
        self.linear_relu_stack: nn.Sequential = nn.Sequential(
            nn.Linear(
                in_features=28 * 28,
                out_features=512,
                bias=True,
                device=device,
                dtype=torch.float32,
            ),
            nn.ReLU(inplace=False),
            nn.Linear(
                in_features=512,
                out_features=512,
                bias=True,
                device=device,
                dtype=torch.float32,
            ),
            nn.ReLU(inplace=False),
            nn.Linear(
                in_features=512,
                out_features=10,
                bias=True,
                device=device,
                dtype=torch.float32,
            ),
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.flatten(x)
        logits: Tensor = self.linear_relu_stack(x)
        return logits


model: NeuralNetwork = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in
PyTorch](buildmodel_tutorial.html).


---


# Optimizing the Model Parameters

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [8]:
loss_fn: CrossEntropyLoss = CrossEntropyLoss()
optimizer: SGD = SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training
dataset (fed to it in batches), and backpropagates the prediction error
to adjust the model\'s parameters.


In [16]:
def train(
    dataloader: DataLoader, model: nn.Module, loss_fn: _Loss, optimizer: Optimizer
) -> None:
    size: int = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X: Tensor
        y: Tensor
        X, y = X.to(device), y.to(device)

        pred: Tensor = model(X)
        loss: Tensor = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            current: int = (batch + 1) * len(X)
            loss: float = loss.item()
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model\'s performance against the test dataset to
ensure it is learning.


In [19]:
def test(dataloader: DataLoader, model: nn.Module, loss_fn: _Loss) -> None:
    size: int = len(dataloader.dataset)
    num_batches: int = len(dataloader)

    model.eval()
    test_loss: float = 0.0
    correct: float = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            X: Tensor
            y: Tensor
            X, y = X.to(device), y.to(device)

            pred: Tensor = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n\tAccuracy: {(100 * correct):>4f}% \n\tAvg loss: {test_loss:>8f} \n"
    )

The training process is conducted over several iterations (_epochs_).
During each epoch, the model learns parameters to make better
predictions. We print the model\'s accuracy and loss at each epoch;
we\'d like to see the accuracy increase and the loss decrease with every
epoch.


In [20]:
epochs: int = 10
for _ in range(epochs):
    print(f"Epoch {_ + 1} \n{'-' * 100}")

    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1 
----------------------------------------------------------------------------------------------------
loss: 2.092669  [   64/60000]
loss: 2.079964  [ 6464/60000]
loss: 2.001020  [12864/60000]
loss: 2.019057  [19264/60000]
loss: 1.943412  [25664/60000]
loss: 1.887611  [32064/60000]
loss: 1.901937  [38464/60000]
loss: 1.819665  [44864/60000]
loss: 1.833333  [51264/60000]
loss: 1.736687  [57664/60000]
Test Error: 
	Accuracy: 52.646667% 
	Avg loss: 1.740782 

Epoch 2 
----------------------------------------------------------------------------------------------------
loss: 1.796459  [   64/60000]
loss: 1.762834  [ 6464/60000]
loss: 1.632664  [12864/60000]
loss: 1.673879  [19264/60000]
loss: 1.556913  [25664/60000]
loss: 1.535893  [32064/60000]
loss: 1.538823  [38464/60000]
loss: 1.462873  [44864/60000]
loss: 1.495417  [51264/60000]
loss: 1.385324  [57664/60000]
Test Error: 
	Accuracy: 60.370000% 
	Avg loss: 1.404069 

Epoch 3 
-------------------------------------------------------

Read more about [Training your model](optimization_tutorial.html).


---


# Saving Models

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).


In [22]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to `model.pth`.")

del model
gc.collect()

Saved PyTorch Model State to `model.pth`.


3634

# Loading Models

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


In [23]:
model: NeuralNetwork = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.


In [56]:
classes: List[str] = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
rand_idx: int = random.randint(0, len(test_data) - 1)
x: Tensor = test_data.__getitem__(rand_idx)[0]
y: int = test_data.__getitem__(rand_idx)[1]
with torch.no_grad():
    x = x.to(device)

    pred: Tensor = model(x)
    predicted = classes[pred[0].argmax(0)]
    actual = classes[y]
    print(f"Index: {rand_idx} \n\tPredicted: '{predicted}' \n\tActual: '{actual}'")

Index: 23501 
	Predicted: 'Bag' 
	Actual: 'Bag'


Read more about [Saving & Loading your
model](saveloadrun_tutorial.html).
